In [13]:
import pandas as pd
import numpy as np
import pickle

from tqdm import tqdm

from sklearn.metrics import f1_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm

In [14]:
with open('ready2go/x_train_apostles.pickle', 'rb') as f:
    x_train_apostles = pickle.load(f)
with open('ready2go/x_test_apostles.pickle', 'rb') as f:
    x_test_apostles = pickle.load(f)
    
with open('ready2go/x_train_intersect.pickle', 'rb') as f:
    x_train_intersect = pickle.load(f)
with open('ready2go/x_test_intersect.pickle', 'rb') as f:
    x_test_intersect = pickle.load(f)
    
with open('ready2go/x_train_full.pickle', 'rb') as f:
    x_train_full = pickle.load(f)
with open('ready2go/x_test_full.pickle', 'rb') as f:
    x_test_full = pickle.load(f)
    
with open('ready2go/y_train.pickle', 'rb') as f:
    y_train = pickle.load(f)
x_train_apostles = np.array(x_train_apostles)
x_test_apostles = np.array(x_test_apostles)

x_train_intersect = np.array(x_train_intersect)
x_test_intersect = np.array(x_test_intersect)

x_train_full = np.array(x_train_full)
x_test_full = np.array(x_test_full)

y_train = np.array(y_train)

In [15]:
print(x_train_apostles.shape, x_test_apostles.shape)
print(x_train_intersect.shape, x_test_intersect.shape)
print(x_train_full.shape, x_test_full.shape)
print(y_train.shape)

(11690, 12) (16627, 12)
(11690, 25) (16627, 25)
(11690, 25) (16627, 25)
(11690,)


In [16]:
X_train = np.hstack((x_train_apostles, x_train_intersect[:, :]))
X_test = np.hstack((x_test_apostles, x_test_intersect[:, :]))
print(X_train.shape)
print(X_test.shape)

(11690, 37)
(16627, 37)


In [17]:
X_concatenated_full = np.vstack((x_train_full, x_test_full))

scaler = StandardScaler()
X_concatenated_scaled_full = scaler.fit_transform(X_concatenated_full)
X_train_scaled_full = X_concatenated_scaled_full[:x_train_full.shape[0]]
X_test_scaled_full = X_concatenated_scaled_full[x_train_full.shape[0]:]
X_train_scaled_full.shape, X_test_scaled_full.shape

((11690, 25), (16627, 25))

In [18]:
X_concatenated = np.vstack((X_train, X_test))

scaler = StandardScaler()
X_concatenated_scaled = scaler.fit_transform(X_concatenated)
X_train_scaled = X_concatenated_scaled[:X_train.shape[0]]
X_test_scaled = X_concatenated_scaled[X_train.shape[0]:]
X_train_scaled.shape[0], X_test_scaled.shape[0]

(11690, 16627)

In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import AdaBoostClassifier

from sklearn import svm

forest = RandomForestClassifier(max_depth=10)
forest_full = RandomForestClassifier(max_depth=10)

regr = ExtraTreesClassifier(max_depth=15)
regr_full = ExtraTreesClassifier(max_depth=15)

log = LogisticRegression()
log_full = LogisticRegression()

ada = AdaBoostClassifier(n_estimators=50, algorithm='SAMME')
ada_full = AdaBoostClassifier(n_estimators=50, algorithm='SAMME')

forest.fit(X_train_scaled, y_train)
regr.fit(X_train_scaled, y_train)
log.fit(X_train_scaled, y_train)
ada.fit(X_train_scaled, y_train)


forest_full.fit(X_train_scaled_full, y_train)
regr_full.fit(X_train_scaled_full, y_train)
log_full.fit(X_train_scaled_full, y_train)
ada_full.fit(X_train_scaled_full, y_train)

from_titles = (forest.predict_proba(X_train_scaled)[:, 1] +\
               regr.predict_proba(X_train_scaled)[: ,1] +\
               log.predict_proba(X_train_scaled)[:, 1] +\
               ada.predict_proba(X_train_scaled)[:, 1])/ 4

from_texts = (forest_full.predict_proba(X_train_scaled_full)[:, 1] +\
              regr_full.predict_proba(X_train_scaled_full)[: ,1] +\
              log_full.predict_proba(X_train_scaled_full)[:, 1] +\
              ada_full.predict_proba(X_train_scaled_full)[:, 1]) / 4

y_pred = (from_titles + from_texts) / 2
res = f1_score(y_train, np.array(y_pred > 0.4, dtype=int))
print(res)

/home/sapun/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/sapun/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/sapun/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/sapun/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/

0.8445176341429944


In [20]:
from_titles = (forest.predict_proba(X_test_scaled)[:, 1] +\
               regr.predict_proba(X_test_scaled)[: ,1] +\
               log.predict_proba(X_test_scaled)[:, 1]) / 3

from_texts = (forest_full.predict_proba(X_test_scaled_full)[:, 1] +\
              regr_full.predict_proba(X_test_scaled_full)[: ,1] +\
              log_full.predict_proba(X_test_scaled_full)[:, 1]) / 3

y_pred = (from_titles + from_texts) / 2
y_pred_test = np.array(y_pred > 0.4, dtype=int)
print(y_pred_test)

[1 1 0 ... 1 1 1]


In [21]:
test_data = pd.read_csv('test_groups.csv')

In [22]:
ans_data = {'pair_id': [], 'target': []}
ans_data['pair_id'] = test_data['pair_id']
ans_data['target'] = np.array(y_pred_test, dtype=int)
df = pd.DataFrame(ans_data, columns = ['pair_id', 'target'])
df.to_csv('my_prediction.csv', index=None)